In [1]:
import pandas as pd
import re
from keybert import KeyBERT
import yfinance as yf
import time
import random
from collections import Counter
from nltk.stem import WordNetLemmatizer
import numpy as np

news_raw = pd.read_csv("C:/projectnasdaq/news_raw.csv", encoding='latin1') # 뉴스 raw data
nasdaq_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks.csv", encoding='latin1') # stock_list data
rep_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks_refine_total.csv", encoding='latin1') # 대표단어 csv

# 테스트 용도로 데이터 1000개
news_raw = news_raw.head(1000)
news_raw = news_raw[['news_id','title','summary']]
nasdaq_stock = nasdaq_stock[['pk','symbol','name']]
rep_stock = rep_stock[['pk','symbol','name','name_a']]

# new_raw 데이터 전처리
news_raw['summary'] = news_raw['summary'].str.lower() #소문자화
news_raw['summary'] = news_raw['summary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())
news_raw['summary'].replace('bloomberg', '', regex=True, inplace=True)

# news_raw 데이터 토큰화
news_raw['tokenize'] = 0
news_raw['tokenize'] = news_raw['summary'].str.split(" ")
news_raw['tokenize'] = news_raw['tokenize'].apply(lambda x: [i for i in x if i != "" and i != " "])

# news raw 데이터에서 필요한 컬럼 생성
news_raw[['lemma_summary','lemma_tokenize','news_keyword_5','news_keyword_10']] = 0

## 회사 이름 추출 => 추출 완료 ( company_word.csv 에 저장 )

In [135]:
# 회사이름 추출 데이터 저장용도
# list에 담아주기
tokenize_list = news_raw['tokenize']
rep_list = rep_stock['name_a'].str.split(",")

company_word = pd.DataFrame(columns=['news_id', 'company_word','pk'])

# 대표단어 csv파일에서 대표단어가 포함되면 맵핑
i = 0

for token_num, token in enumerate(tokenize_list):  # news_raw data 토큰화
    for rep_num, rep in enumerate(rep_list):  # 대표단어 csv파일에서 대표단어에 해당
        i = i + 1
        if len(rep) == 1:
            if rep[0] in token:
                company_word.loc[i] = [news_raw['news_id'][token_num], rep[0], rep_stock['pk'][rep_num]]

        elif len(rep) == 2:  # 대표단어가 2개로 된 단어일 때
            if rep[0] in token:  # 대표단어의 첫번째 단어가 org단어에 있으면
                found = token.index(rep[0])  # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치 번호
                try:
                    search = found + 1  # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = token[search]  # org의 (+1을 한) 다음 단어에 해당
                    if rep[1] == search_found:
                        company_word.loc[i] = [news_raw['news_id'][token_num], rep[0] + " " + rep[1],
                                                 rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

        elif len(rep) == 3:
            if rep[0] in token:
                try:
                    search_found = token[token.index(rep[0]) + 1]
                    if rep[1] == search_found:
                        two_found = token[token.index(rep[0]) + 2]
                        if rep[2] == two_found:
                            company_word.loc[i] = [news_raw['news_id'][token_num],
                                                     rep[0] + " " + rep[1] + " " + rep[2], rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

company_word = pd.merge(company_word, nasdaq_stock, how='left', left_on='pk', right_on='pk')
#company_word.to_csv('C:/projectnasdaq/project_test/company_word.csv')

KeyboardInterrupt: 

## 뉴스 원문에서 키워드 추출 ( Keybert 사용 )=> 추출 완료 ( news_raw.csv 에 저장 )

In [2]:
# WordNetLemmatizer 패키지로 단복수 문제 처리
token = news_raw['tokenize']

for i, j in enumerate(token):
    lemmatizer = WordNetLemmatizer()
    lemma_word = [lemmatizer.lemmatize(word) for word in j]
    news_raw['lemma_tokenize'][i] = lemma_word
    news_raw['lemma_summary'][i] = " ".join(lemma_word)

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [134]:
lemma_summary = news_raw['lemma_summary']
for sum_num, summary in enumerate(lemma_summary):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(summary)
    news_raw['news_keyword_5'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1))
    news_raw['news_keyword_10'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1),top_n= 10)
#news_raw.to_csv('C:/projectnasdaq/project_test/news_raw.csv')

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

KeyboardInterrupt: 

## 종목 정보 수집 ( yfinance 패키지 사용 ) => 수집완료

In [ ]:
# 종목코드 리스트
stocks = nasdaq_stock['symbol']
# 종목코드에 대한 정보 수집용 데이터프레임 생성
stock_info = pd.DataFrame(columns=['symbol', 'info'])
i = 0

for stock_num, stock in enumerate(stocks):
    tickers = yf.Ticker(stock)
    ticker = tickers.info
    i = i + 1
    #print(stock_num,stock, "===>>",ticker)
    stock_info.loc[i] = [stock, ticker]
    time.sleep(random.uniform(3, 4))

#stock_info.to_csv('C:/projectnasdaq/project2/stock_info.csv')

## yahoo_dataset 파일 수정 ( yahoo_dataset_mapping.csv 에 저장 )

In [66]:
# yahoo_dataset 파일 수정 ( -> symbol name 수정 )
yahoo_dataset = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset.csv')
# yahoo_dataset에서 필요없는 컬럼들 지우고 csv 파일에 있는 종목코드에 종목명 데이터들 붙임

yahoo_dataset.insert(1,'new_symbol','0')
yahoo_dataset.insert(2,'name','0')
yahoo_dataset.insert(3,'company_word','0')

yahoo_dataset_symbol = yahoo_dataset['symbol']
rep_stock_symbol = rep_stock['symbol']

for i, j in enumerate(yahoo_dataset_symbol):
    for k, l in enumerate(rep_stock_symbol):
        if j == l:
            yahoo_dataset['new_symbol'][i] = rep_stock['symbol'][k]
            yahoo_dataset['name'][i] = rep_stock['name'][k]
            yahoo_dataset['company_word'][i] = rep_stock['name_a'][k]
            break

# yahoo_dataset 컬럼 정리
yahoo_dataset.drop(columns=["quoteType", "currency", 'regularMarketPrice', 'regularMarketChange', 'regularMarketChangePercent',
             'regularMarketVolume', 'averageDailyVolume3Month', 'marketCap', 'trailingPE', 'fiftyTwoWeekLow',
             'fiftyTwoWeekHigh', 'regularMarketOpen', 'priceHint', 'underlyingSymbol'], inplace=True)
#yahoo_dataset.to_csv('C:/projectnasdaq/project_test/yahoo_dataset.csv')

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

## 추출된 회사 이름이랑 ( 전처리 한 ) 뉴스 키워드 연결 ( company_word.csv 파일에 저장 )


In [3]:
company_word = pd.read_csv('C:/projectnasdaq/project_test/company_word.csv')
news_raw_keyword = pd.read_csv('C:/projectnasdaq/project_test/news_raw.csv') # 실행을 전부 돌리면 시간 오래 걸려서 미리 키워드 뽑아놓고 계속 파일 가져와서 썼기때문에 news_raw_keyword라는 이름으로 가져와서 씀 , news_raw 이름으로 그대로 가져오면 키워드 컬럼이 비워져있는 상태라서 오류 발생

# 키워드 전처리
news_raw_keyword['news_keyword_5'] = news_raw_keyword['news_keyword_5'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())  # 정규식전처리
news_raw_keyword['news_keyword_5'] = news_raw_keyword['news_keyword_5'].apply(lambda x: re.sub(r"\s+", " ", str(x)).strip())  # 공백 여러개 하나로
news_raw_keyword['news_keyword_5'] = news_raw_keyword['news_keyword_5'].str.split(" ")

news_raw_keyword['news_keyword_10'] = news_raw_keyword['news_keyword_10'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())  # 정규식전처리
news_raw_keyword['news_keyword_10'] = news_raw_keyword['news_keyword_10'].apply(lambda x: re.sub(r"\s+", " ", str(x)).strip())  # 공백 여러개 하나로
news_raw_keyword['news_keyword_10'] = news_raw_keyword['news_keyword_10'].str.split(" ")

company_word['stock_keyword_5'] = 0

keyword_list = news_raw_keyword['news_keyword_5']

news_raw_id = news_raw_keyword['news_id']
company_word_id = company_word['news_id']

for i, j in enumerate(company_word_id):
    for k, l in enumerate(news_raw_id):
        if j == l:
            company_word['stock_keyword_5'][i] = news_raw_keyword['news_keyword_5'][k]
#company_word.to_csv('C:/projectnasdaq/project_test/company_word.csv')

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## 종목명이 추출된 뉴스 키워드들끼리 모으기

In [4]:
# company단어들 하나씩만 list에 담기 -> 단어 이름이 같은 종목명들의 키워드 모으기 위해서
company_word = pd.read_csv('C:/projectnasdaq/project_test/company_word.csv')
#company_word.drop(columns=["Unnamed: 0"], inplace=True)
company_words = company_word['company_word']

company_word_same_list = []
for company in company_words:
    if company not in company_word_same_list:
        company_word_same_list.append(company)

company_word_sames = pd.DataFrame(company_word_same_list, columns=['company_word_same'])
company_word_same = company_word_sames['company_word_same']

# 키워드랑 company_name이랑 데이터프레임에서 맵핑 시키기위한 새로운 dataframe 생성
stock_info = pd.DataFrame(columns=['company', 'stock_keyword'])

# company_name에 대한 키워드들 딕셔너리 형태로 모으기
for i, j in enumerate(company_word_same):
    stock_info.loc[i] = [j, company_word[company_word['company_word'] == j]['stock_keyword_5'].tolist()]

# company_name_keyword_organize.csv에 column 삽입
stock_info.insert(0, 'symbol', '0')
stock_info.insert(1, 'name', '0')

yahoo_dataset = pd.read_csv('C:/projectnasdaq/project_test/yahoo_dataset.csv')
company_word_ = company_word['company_word']
company = stock_info['company']

for i,j in enumerate(company_word_):
    for k, l in enumerate(company):
        if j==l:
            stock_info['symbol'][k] = company_word['symbol'][i]
            stock_info['name'][k] = company_word['name'][i]
            break

In [5]:
# 키워드 리스트를 하나의 리스트로 합치기
stock_keyword = stock_info['stock_keyword']
for i, j in enumerate(stock_keyword):
    stock_info['stock_keyword'][i] = ''.join(j)

# 합친 키워드 리스트 전처리
stock_info['stock_keyword'].replace(['\]','\[','\''],'',regex=True, inplace=True)
stock_info['stock_keyword'].replace(',',' ',regex=True, inplace=True)
stock_info['stock_keyword'] = stock_info['stock_keyword'].str.split(" ")
stock_info['stock_keyword'] = stock_info['stock_keyword'].apply(lambda x: [i for i in x if i != "" and i != " "])

# 키워드들 높은 빈도순으로 키워드 추출하기
stock_info[['stock_frequency_keyword_4','stock_frequency_keyword_5']] = 0
frequency_word = stock_info['stock_keyword']

for i, j in enumerate(frequency_word):
    # 대표 키워드 단어 빈도 순으로 4개 추출
    stock_info['stock_frequency_keyword_4'][i] = Counter(j).most_common(4)
#    # 대표 키워드 단어 빈도 순으로 5개 추출
    stock_info['stock_frequency_keyword_5'][i] = Counter(j).most_common(5)

# 4개 뽑아온 키워드 전처리
stock_info['stock_frequency_keyword_4'] = stock_info['stock_frequency_keyword_4'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
stock_info['stock_frequency_keyword_4'] = stock_info['stock_frequency_keyword_4'].str.split(" ")
stock_info['stock_frequency_keyword_4'] = stock_info['stock_frequency_keyword_4'].apply(lambda x: [i for i in x if i != "" and i != " "])
# 5개 뽑아온 키워드 전처리
stock_info['stock_frequency_keyword_5'] = stock_info['stock_frequency_keyword_5'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
stock_info['stock_frequency_keyword_5'] = stock_info['stock_frequency_keyword_5'].str.split(" ")
stock_info['stock_frequency_keyword_5'] = stock_info['stock_frequency_keyword_5'].apply(lambda x: [i for i in x if i != "" and i != " "])

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

## 맵핑 (1) 뉴스 원문에 종목명의 대표 키워드가 포함될 때

In [6]:
# 뉴스 원문에 대표 키워드가 포함될 경우 news_id와 symbol 맵핑(lemmatize)
lemma_tokenize_list = news_raw['lemma_tokenize']
stock_frequency_keyword_5 = stock_info['stock_frequency_keyword_5'] # 키워드 빈도수로 정렬 후 4개만 담아온 리스트
mapping_result = pd.DataFrame(columns=['news_id', 'symbol','name','lemma_summary','stock_frequency_keyword_5'])

a = 0
for i, j in enumerate(lemma_tokenize_list):
    for k,l in enumerate(stock_frequency_keyword_5):
        a = a + 1
        together = set(j)&set(l)
        # 뉴스 원문에 대표 키워드가 4개가 교집합으로 있을 경우
        if len(together)==5:
            mapping_result.loc[a] = [news_raw['news_id'][i], stock_info['symbol'][k], stock_info['name'][k],news_raw['lemma_summary'][i], stock_info['stock_frequency_keyword_5'][k]]

print('뉴스 원문에 대표 키워드가 포함될 경우',len(mapping_result))

뉴스 원문에 대표 키워드가 포함될 경우 508


In [7]:
mapping_result_2 = pd.DataFrame(columns=['news_id', 'symbol','name','lemma_summary','stock_frequency_keyword_5','news_keyword_10'])
news_keyword_10 = news_raw_keyword['news_keyword_10']
stock_frequency_keyword_5 = stock_info['stock_frequency_keyword_5']
# frequency => 종목명의 대표키워드
a = 0
for i,j in enumerate(news_keyword_10):
    for k,l in enumerate(stock_frequency_keyword_5):
        a = a + 1
        together = set(j)&set(l)
        if len(together)>=4:
            mapping_result_2.loc[a] = [news_raw_keyword['news_id'][i], stock_info['symbol'][k], stock_info['name'][k],news_raw_keyword['lemma_summary'][i], stock_info['stock_frequency_keyword_5'][k],news_raw_keyword['news_keyword_10'][i]]

print('뉴스 원문 키워드와 대표 키워드의 교집합이 4개 이상일 때 :',len(mapping_result_2))

뉴스 원문 키워드와 대표 키워드의 교집합이 4개 이상일 때 : 974


In [8]:
mapping_result = mapping_result.drop(columns=['stock_frequency_keyword_5'])
mapping_result_2 = mapping_result_2.drop(columns=['stock_frequency_keyword_5', 'news_keyword_10'])
mapping_result_final = pd.concat([mapping_result,mapping_result_2], ignore_index=True)
overlap = mapping_result_final.drop_duplicates(['news_id','symbol'])

In [9]:
overlap

,news_id,symbol,name,lemma_summary
0,345585,DEO,Diageo plc,nestle sa sailed past ailing consumer good riv...
1,345591,ATSG,Air Transport Services Group Inc,nadine scheiner s effort to travel from her ho...
2,345620,COST,Costco Wholesale Corporation,walmart inc might be one of the few big winner...
3,345620,DG,Dollar General Corporation,walmart inc might be one of the few big winner...
4,345620,BBWI,Bath & Body Works Inc,walmart inc might be one of the few big winner...
...,...,...,...,...
1477,443714,LEN,Lennar Corporation,mizuho financial group inc plan to trim office...
1478,443716,GDRX,GoodRx Holdings Inc,a sharp recovery in health care dealmaking wil...
1479,443719,SPI,SPI Energy Co,in a panel discussion during the travel indust...
1480,443747,PBR,Petroleo Brasileiro S,just one week after the u s federal reserve se...


In [ ]:
""" 화요일에 와서 해야할 일 : shortname가 business summary에 포함되면 short name 지우고, 소문자화 한 후에
     business summary 단복수 처리 후 키워드 다섯개 뽑아내기, 그 후에 키워드 전처리, 뉴스 원문에 키워드 포함시 맵핑 &
      뉴스원문 키워드와 교집합이 있을 경우 맵핑, 전부 끝나면 edgar 제대로 된 데이터 수집하기"""

In [43]:
yahoo_dataset = pd.read_csv('C:/projectnasdaq/project_test/yahoo_dataset.csv')
yahoo_dataset['longBusinessSummary'] = 0
yahoo_dataset['company_info'] = yahoo_dataset['company_info'].fillna("0")
company_info = yahoo_dataset['company_info']

for i,j in enumerate(company_info):
    keys =[]
    values = []
    company_info_list = j.split(', "')
    for info_list in company_info_list:
        pair = info_list.split('":')
        try:
            keys.append(pair[0])
            values.append(pair[1])
            my_dict = dict(zip(keys, values))
            BusinessSummary= my_dict.get('longBusinessSummary')
            yahoo_dataset['longBusinessSummary'][i] = BusinessSummary
        except IndexError: continue

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
yahoo_dataset['longBusinessSummary'] = yahoo_dataset['longBusinessSummary'].str.lower() #소문자화
yahoo_dataset['longBusinessSummary'] = yahoo_dataset['longBusinessSummary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())

yahoo_dataset['shortName'] = yahoo_dataset['shortName'].str.lower() #소문자화
yahoo_dataset['shortName'] = yahoo_dataset['shortName'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())

In [45]:
long_business_summary = yahoo_dataset['longBusinessSummary']
yahoo_dataset['shortName'] = yahoo_dataset['shortName'].fillna("NaN_data")
shortname = yahoo_dataset['shortName']

In [46]:
# 종목명 설명 컬럼에 종목명이 포함되면 지우기
for i,j in enumerate(long_business_summary):
    for k,l in enumerate(shortname):
        if l in j:
            yahoo_dataset['longBusinessSummary'][i] = yahoo_dataset['longBusinessSummary'][k].replace(l,'')
            break

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [48]:
# 불용어 처리
long_business_summary_stopwords = {'does','not','have','company','corp','inc'}
yahoo_dataset['longBusinessSummary'].replace(long_business_summary_stopwords, '', regex=True, inplace=True)

In [49]:
long_business_summary_token = yahoo_dataset['longBusinessSummary'].str.split(" ")
long_business_summary_token = long_business_summary_token.apply(lambda x: [i for i in x if i != "" and i != " "])

for i,j in enumerate(long_business_summary_token):
    lemmatizer = WordNetLemmatizer()
    lemma_word = [lemmatizer.lemmatize(word) for word in j]
    yahoo_dataset['longBusinessSummary'][i] = lemma_word
    yahoo_dataset['longBusinessSummary'][i] = " ".join(lemma_word)

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
yahoo_dataset

,Unnamed: 0,symbol,new_symbol,name,company_word,shortName,longName,company_info,finance_info,longBusinessSummary
0,0,A,A,Agilent Technologies Inc,"agilent,technologies",agilent technologies inc,"Agilent Technologies, Inc.","{""address1"": ""5301 Stevens Creek Boulevard"", ""...","{'language': 'en-US', 'region': 'US', 'quoteTy...",provides application focused solution to the l...
1,1,AA,AA,Alcoa Corporation,"alcoa,corporation",alcoa corporation,Alcoa Corporation,"{""address1"": ""201 Isabella Street"", ""address2""...","{'language': 'en-US', 'region': 'US', 'quoteTy...",together with it subsidiary produce and sell b...
2,2,AAC,AAC,Ares Acquisition Corporation,"ares,acquisition",ares acquisition corporation,Ares Acquisition Corporation,"{""address1"": ""245 Park Avenue"", ""address2"": ""4...","{'language': 'en-US', 'region': 'US', 'quoteTy...",significant operation it intends to effect a m...
3,3,AACG,AACG,ATA Creativity Global,"ata,creativity",ata creativity global,ATA Creativity Global,"{""address1"": ""Building No. 2"", ""address2"": ""Ea...","{'language': 'en-US', 'region': 'US', 'quoteTy...",together with it subsidiary provides education...
4,4,AACI,AACI,Armada Acquisition Corp,"armada,acquisition",armada acquisition corp i,Armada Acquisition Corp. I,"{""address1"": ""2005 Market Street"", ""address2"":...","{'language': 'en-US', 'region': 'US', 'quoteTy...",focus on effecting a merger capital stock exch...
...,...,...,...,...,...,...,...,...,...,...
9001,9001,ZYME,ZYME,Zymeworks Inc,zymeworks,zymeworks inc,Zymeworks Inc.,"{""address1"": ""114 East 4th Avenue"", ""address2""...","{'language': 'en-US', 'region': 'US', 'quoteTy...",discovers develops manufacture and delivers hu...
9002,9002,ZYNE,ZYNE,Zynerba Pharmaceuticals Inc,"zynerba,pharmaceuticals",zynerba pharmaceuticals inc,"Zynerba Pharmaceuticals, Inc.","{""address1"": ""80 West Lancaster Avenue"", ""addr...","{'language': 'en-US', 'region': 'US', 'quoteTy...",operates a a clinical stage specialty pharmace...
9003,9003,ZYXI,ZYXI,Zynex Inc,zynex,zynex inc,"Zynex, Inc.","{""address1"": ""9655 Maroon Circle"", ""city"": ""En...","{'language': 'en-US', 'region': 'US', 'quoteTy...",through it subsidiary design manufacture and m...
9004,9004,TRUE,TRUE,TrueCar Inc,truecar,truecar inc,"TrueCar, Inc.","{""address1"": ""120 Broadway"", ""address2"": ""Suit...","{'language': 'en-US', 'region': 'US', 'quoteTy...",significant operation the intends to effect a ...
